<a href="https://colab.research.google.com/github/msmsm104/Dacon/blob/TIL/sampling/OverSampling_(SMOTETomek)_0616.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import random
import pandas as pd
import numpy as np
import os
import librosa

from tqdm.auto import tqdm

from imblearn.over_sampling import *

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import warnings
warnings.filterwarnings(action='ignore') 

## Load Data

In [3]:
def preprocessing(file_train):
    train_df = pd.read_csv(file_train)
    
    train_x = train_df.drop(columns=['id', 'covid19'])
    train_y = train_df['covid19']
    
    def onehot_encoding(ohe, x):
        # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
        encoded = ohe.transform(x['gender'].values.reshape(-1,1))
        encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
        x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
        return x

    # 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(train_x['gender'].values.reshape(-1,1))
    train_x = onehot_encoding(ohe, train_x)
    
    
    model = MLPClassifier(random_state=41)
    model.fit(train_x, train_y)
    
    pred = model.predict(train_x)
    print(f"accuracy score: {accuracy_score(train_y, pred)}")
    print(confusion_matrix(train_y, pred))
    print(f1_score(train_y, pred))
    

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/train_mfcc_data(32).csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/test_mfcc_data(32).csv')

In [5]:
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']
test_x = test_df.drop(columns=['id'])

In [6]:
def onehot_encoding(ohe, x):
        # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
        encoded = ohe.transform(x['gender'].values.reshape(-1,1))
        encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
        x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
        return x

In [7]:
# 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_x['gender'].values.reshape(-1,1))
train_x = onehot_encoding(ohe, train_x)

test_x = onehot_encoding(ohe, test_x)

In [18]:
from imblearn.combine import *
X_samp, y_samp = SMOTETomek(random_state=0).fit_resample(train_x, train_y)

In [19]:
y_samp.value_counts()

0    3491
1    3491
Name: covid19, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X_samp, y_samp, test_size=0.2, random_state=100)

In [21]:
model = MLPClassifier(random_state=41)
model.fit(X_train, y_train)
    
pred = model.predict(X_val)
print(f"accuracy score: {accuracy_score(y_val, pred)}")
print(confusion_matrix(y_val, pred))
print(f1_score(y_val, pred))

accuracy score: 0.8217609162491052
[[473 226]
 [ 23 675]]
0.8442776735459662


In [22]:
pred = model.predict(test_x)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/sample_submission.csv')
submission['covid19'] = pred
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/submit_0616_SMOTETomek.csv', index=False)